---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [48]:
def answer_one():
        
    # Create the graph using read_edgelist, and tab delimiter
    G = nx.read_edgelist('Employee_Movie_Choices.txt', nodetype=str, delimiter='\t', create_using=nx.Graph())
    
    # Separate the nodes into two partitions
    G.add_nodes_from(employees, bipartite=0)
    G.add_nodes_from(movies, bipartite=1)
    
    return G

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [56]:
def answer_two():
    
    G = answer_one()
    # Add node attributes to each set of nodes
    G.add_nodes_from(employees, type='employee')
    G.add_nodes_from(movies, type='movie')
    
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [59]:
def answer_three():
        
    G = answer_two()
    # Create a weighted projected graph of the employees from G
    P = bipartite.weighted_projected_graph(G, employees)
    
    return P

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [156]:
def answer_four():
    
    def get_name_sets(df):
        """Take in employee movies similarities dataframe and create a dictionary of sets used to check if 
        they exist in employee relationships dataframe"""
        name_sets = {}
        for i, row in df.iterrows():
            name1 = row['Name1']
            name2 = row['Name2']
            name_sets[i] = set([name1, name2])

        return name_sets
    
    def add_common_movies(row):
        """Checks if row of employee relationships df contains two employees that share movie interests, 
        returns the value of movies if they do"""
        name1 = row['Name1']
        name2 = row['Name2']

        # Create set of names in row
        myset = {name1, name2}

        # If myset in dict of sets, then obtain the key (index) value of the dict which will be used to obtain the
        # films in common from the movie similarity df
        if myset in name_sets.values():
            index = list(name_sets.keys())[list(name_sets.values()).index(myset)]
            return df1.iloc[index]['Weight']
        else:
            return 0
        
    P = answer_three()
    # Read in P as dataframe and extract the weight value from the attributes column
    df1 = pd.DataFrame(P.edges(data=True), columns=['Name1', 'Name2', 'Weight'])
    df1['Weight'] = df1['Weight'].apply(lambda x: x['weight'])

    # Read in employee relationships txt file
    G_df = pd.read_csv('Employee_Relationships.txt', 
                       delimiter='\t', header=None, names=['Name1', 'Name2', 'Friendship'])
    
    # Obtain name sets of employee movie similarities df
    name_sets = get_name_sets(df1)
    
    # Create a new weight column in G_df using add_common_movies func
    G_df['Weight'] = G_df.apply(add_common_movies, axis=1)
    
    # Calculate the pearson coefficient between two series
    s1 = G_df['Friendship']
    s2 = G_df['Weight']
    
    p_coeff = s1.corr(s2)
    
    return p_coeff
